<a href="https://colab.research.google.com/github/FranklineMisango/Deep_Learning_Projects/blob/main/LimeLocale_Finance_NLP_Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This project encompasses the LimeLocale's Finance API beta official AI that assists the team in Research in payment patterns in Fintech
#As a starting step, It currently automates the Team's research on Stock and Crypto in China's GBA

In [1]:
#Baseline_Setup_and_dependencies
#Pegasus Tokenizer, NLP for summarization
#BS4 scraps the web for news
!pip install transformers
import requests
import re
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from bs4 import BeautifulSoup
from transformers import pipeline


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 13.6 MB/s eta 0:00:00


In [9]:
#Loading and saving the summarization models  from the Hugging face model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

ValueError: ignored

In [84]:
#Test article summarization for tesla Undercovering in china 
#Used from yFinance#
#TODO-Fix the Issues on r responses and paragraph responses from yahoo finance texts
#TODO --> Mass testing on various URLS
'''

url = "https://www.investopedia.com/buffett-buys-occidental-7229701"
r = requests.get(url)
soup = BeautifulSoup(r.text , 'html.parser')
paragraphs = soup.find_all('p')
text = [paragraph.text for paragraph in paragraphs]
words  = ''.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)
input_ids = tokenizer.encode(ARTICLE, return_tensors = 'pt')
output = model.generate(input_ids,max_length = 55, num_beams = 5 , early_stopping = True)
summary = tokenizer.decode(output[0], skip_special_tokens  = True)
summary

'''

'\n\nurl = "https://www.investopedia.com/buffett-buys-occidental-7229701"\nr = requests.get(url)\nsoup = BeautifulSoup(r.text , \'html.parser\')\nparagraphs = soup.find_all(\'p\')\ntext = [paragraph.text for paragraph in paragraphs]\nwords  = \'\'.join(text).split(\' \')[:400]\nARTICLE = \' \'.join(words)\ninput_ids = tokenizer.encode(ARTICLE, return_tensors = \'pt\')\noutput = model.generate(input_ids,max_length = 55, num_beams = 5 , early_stopping = True)\nsummary = tokenizer.decode(output[0], skip_special_tokens  = True)\nsummary\n\n'

This Code slices focus on the Automation of more than one stock symbol

In [88]:
#Building a News and Sentiment Pipleline
#Test with company stock names --Edited to user preference
monitored_tickers = ['GME' , 'TSLA', 'BTC'] # will be edited to allow user input

#Search for stock news using Google and Yahoo Finance
#TODO--> Figure out how to allow a universal Web scrapping instead of relying solely on Yahoo finance. ie investopedia, Bloomberg LP etc
def search_for_stock_news_urls(ticker):
    search_url = 'https://www.google.com/search?q=yahoo+finance+{}&biw'.format(ticker)
    r = requests.get(search_url)
    soup =  BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'GME': ['/?sa=X&ved=0ahUKEwiJ0P6h1ND9AhU6QTABHblNCZoQOwgC',
  '/search?q=yahoo+finance+GME&ie=UTF-8&gbv=1&sei=78MKZMmcKLqCwbkPuZul0Ak',
  '/search?q=yahoo+finance+GME&ie=UTF-8&source=lnms&tbm=nws&sa=X&ved=0ahUKEwiJ0P6h1ND9AhU6QTABHblNCZoQ_AUIBigB',
  '/search?q=yahoo+finance+GME&ie=UTF-8&source=lnms&tbm=bks&sa=X&ved=0ahUKEwiJ0P6h1ND9AhU6QTABHblNCZoQ_AUIBygC',
  '/search?q=yahoo+finance+GME&ie=UTF-8&source=lnms&tbm=vid&sa=X&ved=0ahUKEwiJ0P6h1ND9AhU6QTABHblNCZoQ_AUICCgD',
  '/search?q=yahoo+finance+GME&ie=UTF-8&source=lnms&tbm=isch&sa=X&ved=0ahUKEwiJ0P6h1ND9AhU6QTABHblNCZoQ_AUICSgE',
  'https://maps.google.com/maps?q=yahoo+finance+GME&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiJ0P6h1ND9AhU6QTABHblNCZoQ_AUICigF',
  '/search?q=yahoo+finance+GME&ie=UTF-8&source=lnms&tbm=shop&sa=X&ved=0ahUKEwiJ0P6h1ND9AhU6QTABHblNCZoQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+GME&ie=UTF-8&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwiJ0P6h1ND9AhU6QTABHblNCZoQpwUIDQ',
  '/search?q=yahoo+finance+GME&ie=UTF-8&sou

In [89]:
#Strip out the unwanted URLs--> policies,accounts, preferences etc
excluded_list = ['maps', 'policies', 'preferences', 'support', 'accounts']
def strip_unwanted_urls(urls, excluded_list):
    final_val = []
    for url in urls:
        if 'https://' in url and not any(excluded_word in url for excluded_word in excluded_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            final_val.append(res)
    return list(set(final_val))

cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], excluded_list) for ticker in monitored_tickers}
cleaned_urls


{'GME': ['https://finance.yahoo.com/quote/GME/profile/',
  'https://finance.yahoo.com/quote/gme/news/',
  'https://ca.finance.yahoo.com/chart/GME/',
  'https://finance.yahoo.com/quote/GME/history/',
  'https://finance.yahoo.com/quote/GME/financials/',
  'https://finance.yahoo.com/chart/GME/',
  'https://ca.finance.yahoo.com/quote/GME/',
  'https://finance.yahoo.com/quote/GME/sec-filing/',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26pccc%3D1',
  'https://finance.yahoo.com/quote/GME/options/',
  'https://sg.finance.yahoo.com/quote/gme/',
  'https://finance.yahoo.com/quote/GME/community%3Fp%3DGME',
  'https://finance.yahoo.com/quote/GME/',
  'https://finance.yahoo.com/quote/GME/options%3Fp%3DGME'],
 'TSLA': ['https://www.google.com/search?q%3Dyahoo%2Bfinance%2BTSLA%26pccc%3D1',
  'https://finance.yahoo.com/quote/TSLA/',
  'https://finance.yahoo.com/quote/TSLA%3Fp%3DTSLA',
  'https://finance.yahoo.com/quote/TSLA/financials/',
  'https://finance.yahoo.com/quote/TSLA/history

In [90]:
#Seach and scrape cleaned URLs
def scrape_and_process(urls):
    ARTICLES = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ''.join(text).split(' ')[:400]
        ARTICLE = ''.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'GME': ['Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Weareexperiencingsometemporaryissues.Themarketdataonthispageiscurrentlydelayed.Pleasebearwithusasweaddressthisandrestoreyourpersonalizedlists.',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  '©2023-Privacy-Terms',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Thankyouforyourpatience.Ourengineersareworkingquicklytoresolvetheissue.',
  'Weareexperiencingsometemporaryissues.Themarketd

In [1]:
#Refer to the solo article solo testing section to understand the model
def summarize_all_articles(articles):
    summaries = []
    for article in articles:
        input_ids = model.tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = model.tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries
 
summaries = {ticker:summarize_all_articles(articles[ticker]) for ticker in monitored_tickers}
summaries

NameError: ignored

In [ ]:
#Adding Sentiment Analysis to give a Negative/Positive score
#-VE means the market is not good for the product
#+VE means the market is good for the product/investment
from transformers import pipeline
sentiment = pipeline('sentiment analysis')

scores ={ticker:summarize_all_articles(articles[ticker]) for ticker in monitored_tickers}
scores

KeyError: ignored

In [ ]:
#Exporting the results to CSV